In [1]:
#Librerias backend:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.optimizers import RMSprop


# Librerias frontend:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, Flatten, MaxPooling2D
import matplotlib.pyplot as plt

Using TensorFlow backend.


# Presentacion del dataset y objetivo de la convnet:

Se tiene un dataset contiendo imagenes de 16 piezas de lego distintas. El objetivo es lograr predecir al nombre de la pieza mostrada en la imagen.

In [2]:
#plt.imshow("/content/drive/My Drive/MLII/CNN/LEGO brick images/train/11214 Bush 3M friction with Cross axle/201706171006-0001.png")

Para la implementacion del modelo se utilizará Keras, dada la simplicidad de implementacion y la flexibilidad en cuanto a la eleccion de la arquitectura.

# Architectura de la CNN:

Se tomaran en cuenta las recomendaciones hechas en clase:

* Se utilizara la funcion de activacion ReLu.
* Se Utilizará Max Pooling (lo que hay "mas importante")
* No se utiizará **dropaut**
* Dado que el dataset consiste en imagenes renderizadas de piezas de lego rotadas, se evaluará la opcion de realizar aumentado de datos.
* Se utilizará el optimizador Adam 
* Se utilizará un batch size de 64

Las imagenes tiene un tamaño estandard de 72x72, siendo cada una de 3 canalaes. Por lo tanto, mi capa de entrada será de: 72x72x3.

La capa convolucional

# Carga del dataset y preprocesamiento de las imagenes.

Fuente: https://keras.io/preprocessing/image/


In [3]:
imageWidth = 72
imageHeight = 72 
imageChannel = 3
testPorcen = 0.15
batchSize = 32
dataPath = 'train'

Declaracion del generador para preprocesar las imagenes. Se tomara un 15% de datos para test:

In [4]:
#Se utilizaran los valores por defecto de la generador:
train_datagen = ImageDataGenerator(
        validation_split=testPorcen, #Split argument for data division.
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

Procedemos con la separacion de la data en Train y Test. **Debido a que especificamos el valor de validation_split en el generador anterior, ahora debemos especificar que datos son de validacion (test) y cuales son de train.**:

In [5]:
train_generator = train_datagen.flow_from_directory(
        dataPath,
        target_size=(imageHeight, imageWidth),
        batch_size=batchSize,
        class_mode='categorical', #Por defecto es categorical, sin embargo, no esta de más settearla manualmente.
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        dataPath,
        target_size=(imageHeight, imageWidth),
        batch_size=batchSize,
        class_mode='categorical', #Por defecto es categorical, sin embargo, no esta de más settearla manualmente.
        subset='validation')

Found 5423 images belonging to 16 classes.
Found 956 images belonging to 16 classes.


Procedemos a aplicar los generadores para obtener nuestra deta:

# Creacion del modelo:

Fuente: https://towardsdatascience.com/boost-your-cnn-image-classifier-performance-with-progressive-resizing-in-keras-a7d96da06e20

In [6]:
#Valores de la capa convolucional:
filtersNumber = 8
filterSize = 5
pool_size = 3
fully_connected_neurons = 64
drop_out_factor = 0.6
classes_out = 16 #numero de tipos de legos (neuronas de salida.)
numEpochs = 20

In [9]:
model = Sequential()
#Primera capa de convolución (se estan tomando en cuenta todas las recomendaciones indicadas: Funcion de activacion Relu, Mini Batch de 64)
model.add(Conv2D(batchSize, input_shape=(imageHeight, imageWidth, imageChannel) , activation='relu', kernel_size=(filterSize, filterSize)  ))
#Pooling, tomando en cuenta la recomendacion de utilizar MaxPooling:
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))


#Segunda capa de convolucion:
model.add(Conv2D(batchSize, input_shape=(imageHeight, imageWidth, imageChannel) , activation='relu', kernel_size=(filterSize, filterSize)  ))
#Pooling para la segunda capa:
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))


#Aplanado de la ultima capa de pooling con las respectivas caracteristicas para la parte "fully connected":
model.add(Flatten())
#Se utilizaran 64 neuronas en la etapa fully connected
model.add(Dense(fully_connected_neurons, activation='relu'))
#En esta etapa si se utilizará dropout, ya que equivale a un modelo MLP:
model.add(Dropout(rate=drop_out_factor))
#Se tienen 16 neuronas de salida, 1 para cada posible valor de la salida. Se utiliza Softmax puesto que necesitamos una probabiliad
model.add(Dense(classes_out, activation='softmax'))

model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 68, 68, 32)        2432      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 18, 18, 32)        25632     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                73792     
_____

In [10]:
#Compilando el modelo:
model.compile(
    optimizer=RMSprop(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
#Entrenando el modelo:
preddicted = model.fit_generator(
    train_generator,
    use_multiprocessing=True,
    steps_per_epoch=len(train_generator.filenames) // batchSize,
    epochs=numEpochs,
    validation_data=validation_generator,
    validation_steps=len(train_generator.filenames) // batchSize
    )

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
169/169 [==============================] - 37s 217ms/step - loss: 2.3320 - acc: 0.2234 - val_loss: 1.7630 - val_acc: 0.3692
Epoch 2/20
169/169 [==============================] - 42s 246ms/step - loss: 1.8791 - acc: 0.3479 - val_loss: 1.4978 - val_acc: 0.4387
Epoch 3/20
169/169 [==============================] - 41s 242ms/step - loss: 1.6722 - acc: 0.4065 - val_loss: 1.3927 - val_acc: 0.4584
Epoch 4/20
169/169 [==============================] - 43s 255ms/step - loss: 1.4885 - acc: 0.4556 - val_loss: 1.2734 - val_acc: 0.5082
Epoch 5/20
169/169 [==============================] - 50s 295ms/step - loss: 1.4165 - acc: 0.4802 - val_loss: 1.2218 - val_acc: 0.5325
Epoch 6/20
169/169 [==============================] - 55s 324ms/step - loss: 1.3247 - acc: 0.5120 - val_loss: 1.1724 - val_acc: 0.5702
Epoch 7/20
169/169 [==============================] - 51s 303ms/step - loss: 1.2440 - acc: 0.5366 - val_loss: 1.1320 - val_acc: 0.5654
Epoch 8

In [12]:
xTestBatch, yTestBatch = validation_generator.next()
np.mean(1.0*(model.predict_classes(xTestBatch) == np.argmax(yTestBatch,axis=1)))

0.84375

In [17]:
listBatch = []
for i in range(10):
    xTestBatch, yTestBatch = validation_generator.next()
    accuracy = np.mean(1.0*(model.predict_classes(xTestBatch) == np.argmax(yTestBatch,axis=1)))
    listBatch.append(accuracy)
    print("Batch sample: "+ str(i)+" accuracy: "+ str(accuracy))
print(np.mean(listBatch))

Batch sample: 0 accuracy: 0.8125
Batch sample: 1 accuracy: 0.78125
Batch sample: 2 accuracy: 0.71875
Batch sample: 3 accuracy: 0.875
Batch sample: 4 accuracy: 0.71875
Batch sample: 5 accuracy: 0.78125
Batch sample: 6 accuracy: 0.65625
Batch sample: 7 accuracy: 0.625
Batch sample: 8 accuracy: 0.71875
Batch sample: 9 accuracy: 0.65625
0.734375


# Se obtiene un accuracy del 73% en promedio, por lo que se modificara el modelo.

Hiperparametros a resaltar:
* Batch size de 32
* Tamaño de los filtros en las capas convolucionales: 5x5
* Epochs: 20
* Optimizador: RMSprop